This script generates the ETCCDI precipitation indices from the MCB 5% seeding runs. 

Outputs one file per index:

* Rx1Day  
* Rx5Day  
* PTOT   
* sdii  
* r10  
* r20   
* Q95  (mean threshold over 30 years, permuted by year)  
* Q99  (mean threshold over 30 years, permuted by year)
* CWD   
* CDD  

P95Tot (proportion of annual total), P95T etc. are calculated separately from the thresholds created for SSP245. Saved in /glade/work/maritye/Data/ARISE-SAI/ETCCDI/SSP245/

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np # data arrays
import xarray as xr # data array manipulation
import pandas as pd
import datetime as dt
import os

Define the precipitation functions

In [3]:
from importlib import reload
import config
import precex_func
import utils #pyclimdex by B Groenks


Read in each of the datasets in turn, calculate indices, then write the output to netcdf.

In [4]:
iDir = "/glade/work/maritye/Data/ARISE-MCB/Dailies/PRECT/"
oDir = "/glade/work/maritye/Data/ARISE-MCB/ETCCDI/PRECT/"

Create standard information to include with each data set

In [17]:
years = np.arange(2035,2070)
dates= '2035-2069'
members=range(10)
lat = np.linspace(-90,90,num=192)
lon = np.linspace(0,358.75,num=288)
dims = ('member','year', 'lat', 'lon')
coords = dict(member = members, year=years, lat=lat, lon=lon)
attribs = dict(description='Precipitation Indices based on ETCCDI definitions. MCB scenarios with 5% global seeding', 
                history='Created by Mari Tye November 2023.' ),


Create some blank arrays to fill with each index for each ensemble member. Write these out at the end with netcdf

In [19]:
Rx1d = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='RX1D')
Rx5d = xr.DataArray(None, coords=coords,  dims=dims, attrs=attribs, name='RX5D')
PTOT = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='PRCPTOT')
sdii = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='SDII')
r10 = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='R10mm')
r20 = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='R20mm')
CWD = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='CWD')
CDD = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='CDD')
N95 = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='N95')
N99 = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='N99')
P95 = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='P95')
P99 = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='P99')
P95T = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='P95T')
P99T = xr.DataArray(None, coords=coords, dims=dims, attrs=attribs, name='P99T')

read in thresholds from SSP245m take the mean of the ensemble as the threshold for these runs.

In [14]:
with xr.open_dataset('/glade/work/maritye/Data/ARISE-SAI/ETCCDI/SSP245/PRECT/b.e21.BWSSP245cmip6.f09_g17.CMIP6-SSP2-4.5-WACCM.cam.h1.Q95.2015-2069.nc') as ds95:
    dp95 = ds95.mean(dim='member')

with xr.open_dataset('/glade/work/maritye/Data/ARISE-SAI/ETCCDI/SSP245/PRECT/b.e21.BWSSP245cmip6.f09_g17.CMIP6-SSP2-4.5-WACCM.cam.h1.Q99.2015-2069.nc') as ds99:
    dp99 = ds99.mean(dim='member')

List all of the precipitation input files

In [11]:
files = os.listdir(iDir)

Calculate all of the indices

In [ ]:
for eff in range(10): 
    print(files[eff])
    #### Open file
    fp = os.path.join(iDir, files[eff])
    ds = xr.open_dataset(fp)
    ds = ds.sel(time=slice('2015-01-01', '2069-12-31'))
    
    #### Calculate indices
#Annual maximum, total and mean

    Rx1d[eff,:,:,:] = precex_func.yearly_rx1day(ds)
    print('rx1d')
    Rx5d[eff,:,:,:] = precex_func.yearly_rx5day(ds)
    print('rx5d')
    foo = precex_func.prcptot(ds)
    PTOT[eff,:,:,:] = foo
    print('ptot')

    sdii[eff,:,:,:] = precex_func.sdii(ds)
    print('sdii')

#Fixed threshold count of days

    r10[eff,:,:,:] = precex_func.annual_r10mm(ds)
    print('r10')
    r20[eff,:,:,:]= precex_func.annual_r20mm(ds)
    print('r20')

#Spells

    CWD[eff,:,:,:] = precex_func.cdd(ds)
    print('cwd')
    CDD[eff,:,:,:] = precex_func.cwd(ds)
    print('cdd')
    
#Location specific threshold counts
    N95[eff,:,:,:] = precex_func.annualnum_above_q(ds, dp95)
    print('N95')
    N99[eff,:,:,:] = precex_func.annualnum_above_q(ds, dp99, threshold=0.99)
    print('N99')
    bar = precex_func.annualtot_above_q(ds, dp95)
    P95[eff,:,:,:] = bar
    print('P95')
    boo = precex_func.annualtot_above_q(ds, dp99, threshold=0.99)
    P99[eff,:,:,:] = boo
    print('P99')
"""
    P95T[eff,:,:,:] = bar/foo
    print('P95T')
    P99T[eff,:,:,:] = boo/foo
    print('P99T')
"""
    



b.e21.BSSP245smbb.f09_g17.MCB-050PCT.002.cam.h1.PRECT.20350101-20691231.nc
rx1d
rx5d
ptot
sdii
r10
r20
cwd
cdd
N95
N99
P95
P99
b.e21.BSSP245smbb.f09_g17.MCB-050PCT.009.cam.h1.PRECT.20350101-20691231.nc
rx1d
rx5d
ptot
sdii
r10
r20
cwd
cdd
N95
N99
P95
P99
b.e21.BSSP245smbb.f09_g17.MCB-050PCT.006.cam.h1.PRECT.20350101-20691231.nc
rx1d
rx5d
ptot
sdii
r10
r20
cwd
cdd
N95
N99
P95
P99
b.e21.BSSP245smbb.f09_g17.MCB-050PCT.007.cam.h1.PRECT.20350101-20691231.nc
rx1d
rx5d
ptot
sdii
r10
r20
cwd
cdd
N95
N99
P95
P99
b.e21.BSSP245smbb.f09_g17.MCB-050PCT.003.cam.h1.PRECT.20350101-20691231.nc
rx1d
rx5d
ptot
sdii
r10
r20
cwd
cdd
N95
N99
P95
P99
b.e21.BSSP245smbb.f09_g17.MCB-050PCT.005.cam.h1.PRECT.20350101-20691231.nc
rx1d
rx5d
ptot
sdii
r10
r20
cwd
cdd
N95
N99
P95
P99
b.e21.BSSP245cmip6.f09_g17.CMIP6-MCB-050PCT.000.cam.h1.PRECT.20350101-20691231.nc
rx1d
rx5d
ptot
sdii
r10
r20
cwd
cdd
N95
N99
P95
P99
b.e21.BSSP245smbb.f09_g17.MCB-050PCT.010.cam.h1.PRECT.20350101-20691231.nc
rx1d
rx5d
ptot
sdii
r10
r20
c

In [34]:
Rx1d

<xarray.DataArray 'RX1D' (member: 10, year: 35, lat: 192, lon: 288)>
array([[[[2.3116683735224797, 2.311668066568018, 2.311667913090787,
          ..., 2.3116688339541724, 2.3116686804769415,
          2.3116685269997106],
         [2.398077896259565, 2.394241579395384, 2.394245569803388, ...,
          2.4120825401041657, 2.4069651487934607, 2.4028726784308674],
         [2.3744008103676606, 2.3702653663804085, 2.369778536603917,
          ..., 2.4200122481943254, 2.4016777047108917,
          2.3846119574955082],
         ...,
         [10.21232399125438, 10.498742585696164, 10.76285848284897,
          ..., 10.133333557860169, 9.963278330360481,
          9.914432052937627],
         [8.015326898203057, 8.23032271455304, 8.45381073304452, ...,
          7.358191419370996, 7.5743653269455535, 7.791275925228547],
         [5.4696454981240095, 5.482627830133424, 5.4821655567138805,
          ..., 5.491228080245492, 5.491091792464431, 5.465300864671008]],

        [[1.3880164601687284, 1.3880159997370356, 1.3880159997370356,
          ..., 1.3880175345093448, 1.3880178414638067,
          1.388017227554883],
...
          6.8026996132175555]],

        [[2.3034788284803653, 2.3034788284803653, 2.3034788284803653,
          ..., 2.3034788284803653, 2.3034788284803653,
          2.3034788284803653],
         [1.7483533724771405, 1.7436442306006938, 1.744471933307068,
          ..., 1.8334440653688944, 1.8027229884864937,
          1.771904760516918],
         [2.0639924969145795, 2.075827126191143, 2.0889353095299157,
          ..., 2.023837788556193, 2.038754854538638,
          2.0517856853530247],
         ...,
         [10.984482059939182, 11.277441853962955, 11.636369845291483,
          ..., 10.167937148253259, 10.489692340343026,
          10.721840772021096],
         [9.265333869734604, 9.379659059050027, 9.483731969339715, ...,
          8.842782199280919, 8.982008148450404, 9.131006913776218],
         [7.083022705955955, 7.082990168782999, 7.082993238327617, ...,
          7.083006130415015, 7.083010427777481, 7.083018408593489]]]],
      dtype=object)
Coordinates:
  * member   (member) int64 0 1 2 3 4 5 6 7 8 9
  * year     (year) int64 2035 2036 2037 2038 2039 ... 2065 2066 2067 2068 2069
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    description:  history

batch create file names

In [36]:
fileintro = 'b.e21.BSSP245smbb.f09_g17.MCB-050PCT.001-010.cam.h1.'

rx1nm = os.path.join(oDir , (fileintro + 'RX1D.' + dates +'.nc'))
rx5nm = os.path.join(oDir , (fileintro + 'RX5D.' + dates +'.nc'))
ptotnm = os.path.join(oDir , (fileintro + 'PRCPTOT.' + dates +'.nc'))
sdnm = os.path.join(oDir , (fileintro + 'SDII.' + dates +'.nc'))
r10nm = os.path.join(oDir , (fileintro + 'R10mm.' + dates +'.nc'))
r20nm = os.path.join(oDir , (fileintro + 'R20mm.' + dates +'.nc'))
cddnm = os.path.join(oDir , (fileintro + 'CDD.' + dates +'.nc'))
cwdnm = os.path.join(oDir , (fileintro + 'CWD.' + dates +'.nc'))
n95nm = os.path.join(oDir , (fileintro + 'N95.' + dates  + '.nc'))
n99nm = os.path.join(oDir , (fileintro + 'N99.' + dates  + '.nc'))
p95nm =os.path.join(oDir , (fileintro + 'P95.' + dates  + '.nc'))
p99nm = os.path.join(oDir , (fileintro + 'P99.' + dates  + '.nc'))
#p95tnm = os.path.join(oDir , (fileintro + 'P95T.' + dates  + '.nc'
#p99tnm = os.path.join(oDir , (fileintro + 'P99T.' + dates  + '.nc'

and write out to netcdf

In [37]:
Rx1d.to_netcdf(rx1nm)
Rx5d.to_netcdf(rx5nm)
PTOT.to_netcdf(ptotnm)
sdii.to_netcdf(sdnm)
r10.to_netcdf(r10nm)
r20.to_netcdf(r20nm)
CWD.to_netcdf(cwdnm)
CDD.to_netcdf(cddnm)
N95.to_netcdf(n95nm)
N99.to_netcdf(n99nm)
P95.to_netcdf(p95nm)
P99.to_netcdf(p99nm)
#P95T.to_netcdf(p95tnm)
#P99T.to_netcdf(p99tnm)
